In [1]:
import os.path
from sklearn.feature_extraction.text import CountVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from standard_function import *
from return_detail_icd_code import *
import pickle
import unicodedata
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

# Load data, Models and wb File

In [2]:
#load_data_check_default = pd.read_excel("dataset.xls", sheetname="Sheet1", header = None)
load_data_check_default = pd.read_excel("dataset_train.xls", header = None, encoding = 'utf-8')

load_data_check = load_data_check_default.copy()
load_data_check[2] = load_data_check[2].apply(delete_special_char)

In [3]:
#load_data_check[2]

In [4]:
f = open('Eclaim_predict.pickle', 'rb')       # load model dự đoán mã icd 
classifier = pickle.load(f)
f.close()

In [5]:
with open ('exacly_2_word', 'rb') as fp_2:    # Load data về 2 cụm từ 
    words_2 = pickle.load(fp_2)

In [6]:
with open ('exacly_3_word', 'rb') as fp_3:    # Load data về 3 cụm từ
    words_3 = pickle.load(fp_3)

In [7]:
with open('link_words.pkl', 'rb') as f:
    listview = pickle.load(f)

In [8]:
model_2 = Word2Vec.load("2_gram.model")
model_3 = Word2Vec.load("3_gram.model")

In [9]:
def try_except_2_gram ( string):
    try :  
        model_2.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

def try_except_3_gram ( string):
    try :  
        model_3.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

In [10]:
#text = input("Cung cấp chuỗi cần xác định :")
text = 'sốt cao rét run'

text = revert_text(out_special_char(text))    # hàm loại bỏ dấu ngoặc tròn, vuông , \ , /
pr = spaces_num(char_num(text))    # loại bỏ khoản trắng vô nghĩa  
pr1 = ' '.join(pr)                 # convert  list to string
pr1 = delete_special_char(pr1)     # delete others special char --- > string return string

In [11]:
text1 = pr1.strip("").split(" ")       # Convert str to list
text1 = spaces_num(text1)                # loại bỏ khoản trắng thừa trong list
text2 = pr1.strip("").split("*")       # Convert str to list Text 1 chứa list các từ đơn
text2 = spaces_num(text2)                # Text2 chứa List cả cụm từ
temp_02 = text1.copy()

In [12]:
check_logic = True
if (len(text1) >= 3):
    sta = incorrect_spelling(text1, words_2, words_3)
    text1 = sta.strip(' ').split(' ')
    text2 = sta.strip(' ').split('*')
elif len(text1) == 2:
    kq = check_incorrect_2_word(text1,words_2)
    text1 = kq[0]
    check_logic = kq[1]
if len(text1) == 1:
    pass
text1

['sốt', 'cao', 'rét', 'run']

In [13]:
new_str =''
for i in range(0,len(text1)-1):
    new_str += text1[i] + ' ' + text1[i+1] + ","

In [14]:
new_str = new_str.strip(',').split(",")
addr = []
chuoi = ''
for index,i in enumerate (new_str[:len(new_str)]):
    if try_except_2_gram(i) == False :
        addr.append(index)
        chuoi += i + ","
if len(text1)-len(addr) == 1:
    addr.append(max(addr) + 1)

In [15]:
if len(addr) == 0:
    addr.append(-1)
sum_index = addr[0]
temp = []
for i in range(0,len(addr)-1):
    if (addr[i+1] - addr[i]) == 1:
        sum_index = addr[i+1]
    else:
        temp.append(sum_index)
        sum_index = addr[i+1]
temp.append(sum_index)
if temp[0] == '':
    temp.append(len(text1))
else:
    temp = list(set(temp))
    temp.sort(reverse = False)

In [16]:
check = ''
emty = ''
sta = []
sto = []
sta1 = []
for index, x in enumerate(text1):     # cắt chuỗi bệnh thành các loại bệnh bước 1
    if index in temp:
        check += x + ','
    else:
        check += x + ' '
check = spaces_num(check.strip().split(','))

if check_logic == False:             # Tách 2 bệnh chỉ có 2 chữ cái
    check = temp_02

for i in range(0, len(check)):       # Gộp các từ đơn lẻ tránh lỗi Vocabulary
    if len(check[i]) == 1:
        check.remove(check[i])
        break

for i in range(0, len(check)): 
    if len(check[i]) == 1:
        check[i-1] = check[i-1] + ' ' + check[i]
        check.remove(check[i])
                    
for i in check:                     # Tách các bệnh dính liền thành các loại bệnh bước 2
    ix = i.split(' ')
    for j in listview:
        if j in i:
            jx = j.split(' ')
            for index, val in enumerate(ix[0:len(ix)-2]):
                if (ix[index] == jx[0]) and (ix[index+1]==jx[1]) and (try_except_3_gram(j+' '+ix[index+2]) == False):
                    ix.insert(index+1, ',') 
    ix = ' '.join(ix)
    i = ix
    emty += i +','
check = emty.strip(',').split(',')
check

['sốt cao rét run']

In [17]:
with open ('sample_vector', 'rb') as fp:
    itemlist = pickle.load(fp)

In [18]:
count = CountVectorizer()
count.fit(itemlist)
bag_of_words = count.transform(itemlist)

In [19]:
x_new1 = count.transform(check)
x_new1
kq = classifier.predict(x_new1)
kq

array(['B54'], dtype='<U3')

In [20]:
Sam2Data ={"Infor":[]}
for g in range(0,len(kq)):
    df_end = (load_data_check[load_data_check[1] == kq[g]])
    io = decission_value_return_01(df_end, check[g].split(','))
    
    Sam2Data["Infor"].append(io)
Sam2Data

{'Infor': [['B54', 1.0, 'sốt cao rét run']]}